# TPMSS Analysis
## Tire pressure monitor data extraction
**DISCLAIMER:** Use for good, not evil. If you use this for anything unethical/illegal, I don't want to know! As always ethical/legal additions to this are welcomed!


From what I understand, when a car equipped with one of these TPMSS devices is traveling at 20+ mph it will beacon out it's ID, pressure, and temperature at an interval of 60 seconds at 315MHz. What we're focusing on here is the ID, because it's what's most likely to be used to track you in street-level surveillence. 

Potential use cases: 
* Checking your tire pressure. Duh.
* Diagnosing problems with any indicators on your car regarding tire pressure
* (UNETHICAL) Track people's behavior patterns and movements.
* Mass data collection (ANONYMIZE YOUR DATA!)


**by Brian Lechthaler**

*Authored 05/04/19 -- Data cleaned 05/09/19*

In [1]:
#We'll need this to parse and manipulate our data
import pandas as pd
#We will also need numpy to do some mathematics
import numpy as np
#Just in case we need it, here's matplotlib 
import matplotlib.pyplot as plt

## Raw data
As you can see, the raw CSV output from rtl_433 contains a ton of NaN data we need to clip out.

So, let's fix that by cleaning things up a bit and removing all columns containing NaN values.

In [2]:
#Run rtl_433 with args: -f 315M -v -F csv:tpmss_samples0.csv
#make sure the output from that command is in the same folder as this notebook!
csvdata = pd.read_csv("tpmssData_cleaned.csv")
print(csvdata)
print("This data is pretty messy...")

                   time  msg  codes   model  button        id  channel  \
0   2019-05-01 18:00:56  NaN    NaN  Toyota     NaN  00004e2e      NaN   
1   2019-05-01 18:02:20  NaN    NaN  Toyota     NaN  11114be1      NaN   
2   2019-05-01 18:02:32  NaN    NaN  Toyota     NaN  00004e2e      NaN   
3   2019-05-01 18:03:06  NaN    NaN  Toyota     NaN  111151f1      NaN   
4   2019-05-01 18:03:56  NaN    NaN  Toyota     NaN  11114be1      NaN   
5   2019-05-01 18:04:06  NaN    NaN  Toyota     NaN  00004e2e      NaN   
6   2019-05-01 18:04:41  NaN    NaN  Toyota     NaN  111151f1      NaN   
7   2019-05-01 18:05:32  NaN    NaN  Toyota     NaN  11114be1      NaN   
8   2019-05-01 18:05:40  NaN    NaN  Toyota     NaN  00004e2e      NaN   
9   2019-05-01 18:07:09  NaN    NaN  Toyota     NaN  11114be1      NaN   
10  2019-05-01 18:07:13  NaN    NaN  Toyota     NaN  00004e2e      NaN   
11  2019-05-01 18:07:53  NaN    NaN  Toyota     NaN  111151f1      NaN   
12  2019-05-01 18:08:45  NaN    NaN  T

In [3]:
#Now let's clean up our data a little bit
cleandata = csvdata.dropna(axis = 1)
df0 = pd.DataFrame(cleandata)
print(cleandata)
print("See? Much better!")

                   time   model        id  temperature_C  mic  status  type  \
0   2019-05-01 18:00:56  Toyota  00004e2e           22.0  CRC     128  TPMS   
1   2019-05-01 18:02:20  Toyota  11114be1           18.0  CRC     128  TPMS   
2   2019-05-01 18:02:32  Toyota  00004e2e           22.0  CRC     128  TPMS   
3   2019-05-01 18:03:06  Toyota  111151f1           19.0  CRC     128  TPMS   
4   2019-05-01 18:03:56  Toyota  11114be1           18.0  CRC     128  TPMS   
5   2019-05-01 18:04:06  Toyota  00004e2e           22.0  CRC     128  TPMS   
6   2019-05-01 18:04:41  Toyota  111151f1           19.0  CRC     128  TPMS   
7   2019-05-01 18:05:32  Toyota  11114be1           18.0  CRC     128  TPMS   
8   2019-05-01 18:05:40  Toyota  00004e2e           22.0  CRC     128  TPMS   
9   2019-05-01 18:07:09  Toyota  11114be1           18.0  CRC     128  TPMS   
10  2019-05-01 18:07:13  Toyota  00004e2e           22.0  CRC     128  TPMS   
11  2019-05-01 18:07:53  Toyota  111151f1           

## Unique IDs
* These are the unique identification strings for the tire pressure monitors we picked up.
* 4 Hex bits (8 characters): First three bits seem to be unique to the vehicle, last bit seems to indicate which tire it's attached to
* **NB: This data has been anonymized!** There were two cars (assumption) in the csv, the first 2 bits were changed from their raw values. The first car is 0000, and the second car is 1111.


In [4]:
uniqueIDs = pd.unique(df0.id)
print("Here are thte unique IDs from the captured TPMSS data:")
print(uniqueIDs)

Here are thte unique IDs from the captured TPMSS data:
['00004e2e' '11114be1' '111151f1' '11114c50' '00004e42']
